In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
# import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
train = pd.read_csv('train_preprocessing34_full.csv')

train.shape
train.head()

(278894, 266)

,user_context_trade_count,user_city_trade_count,user_day_count,user_shop_count,city_item_count_his,shop_item_count_his,user_shop_next_time_sub,user_age_level,city_item_count,shop_star_level,user_trade_hot,user_item_num,shop_trade_hot,user_count,shop_score_delivery,user_item_last_time_sub,city_hot,user_sell_trade_power_mean,is_last_click,user_is_his,is_last_user_item_click,item_hot,user_item_count_first_time_sub,user_sell_power,user_shop_day_num,item_trade_hot,item_sales_level,item_occupation_trade_count,item_gender_count,minute,hour,item_score2,user_minute_query,user_id_idx,item_pred_property_score,brand_item_count_his,user_minute_count,pred_property_split_count,item_occupation_count,user_shop_trade_count,item_brand_id_idx,next_time_sub,user_day_query,user_sell_power_mean_item_minus,yesterday,user_item_trade_count,user_yesterhour_query,brand_is_his,item_pred_category_score,user_brand_count,user_item_count,city_brand_count_his,item_gender_trade_count,user_shop_count_first_time_sub,brand_hot,item_trade_percent,user_is_trade_his,item_collected_level,item_id_idx,user_sell_power_mean,brand_is_trade_his,city_trade_hot,user_gender_id,city_brand_count,item_minute_query,item_is_his,item_pred_property_score_item%,occupation_hot,context_page_id,shop_id_idx,user_context_count,last_time_sub,item_property_split_count,item_hour_query,user_shop_last_time_sub,user_trade_percent,min_time,shop_item_count,user_count_first_time_sub,user_item_day_query,user_sell_trade_power,shop_hot,user_occupation_id,user_brand_trade_count,brand_trade_hot,user_shop_num,user_item_minute_query,item_city_id_idx,day,item_pv_level,user_hot,user_item_next_time_sub,item_category_split_count,shop_score,brand_item_count,user_hour_query,pred_category_split_count,item_score,user_hour_count,brand_trade_percent,item_price_level,item_age_count,user_yesterday_query,shop_score_service,occupation_trade_hot,user_city_count,item_score3,user_item_day_num,item_pred_property_score_pred%,item_is_trade_his,item_pred_category_score_item%,occupation_trade_percent,shop_review_num_level,shop_review_positive_rate,user_sell_trade_power_mean_item_minus,user_item_hour_query,shop_score_description,yesterhour,max_click_time_sub,is_home,item_pred_category_score_pred%,item_age_trade_count,user_star_level,item_day_query,max_user_item_click_time_sub,user_query_maxsell,category_2,property_7,property_1,query_col_6,item_collected_level_bayes_rate_1,item_price_stair,user_query_maxpv,user_query_cheapest,user_occupation_stair,property_5,user_query_best_service,user_age_stair,query_col_5,query_col_1,query_col_0,user_query_maxqueryitem_c_similarity,user_star_stair,item_pv_stair_shop_id_bayes_rate_2,item_sales_stair,property_8,user_query_maxcollect,property_2,user_query_best_description,user_query_maxreview,property_0,property_3,query_col_2,category_1,hour_stair,user_query_best_delivery,query_col_3,user_query_maxshopstar,time_slice_shop_review_num_level_bayes_rate_1,property_6,item_sales_stair_user_age_stair_bayes_rate_1,item_city_id_shop_id_bayes_rate_2,context_page_stair,user_query_maxqueryitem_p_similarity,item_brand_id_shop_review_num_level_bayes_rate_1,gender_filled,query_col_4,property_4,pred_cat_mean_5,pred_cat_mean_14,pred_cat_mean_17,pred_cat_mean_22,shop_item_cnt,item_cat_vec_22,item_cat_vec_2,item_brand_id+hist_hour_ctr_pred_1,item_brand_id+shop_score_description_pred_1,user_gender_id+star_category_min_pred_1,pred_cat_mean_15,item_id_pred_1,item_cat_vec_21,24h_ppt_hot+star_property_max_pred_1,star_category_mean,item_cat_vec_19,item_price_level+item_pred_cat_cos_pred_1,star_property_min,24h_cat_hot,item_cat_vec_5,item_cat_vec_3,item_cat_vec_16,pred_cat_mean_0,star_property_var,shop_review_num_level+shop_item_cnt_pred_1,pred_cat_mean_6,item_pv_level+shop_review_positive_rate_pred_1,item_cat_vec_9,item_sales_level+item_pv_level_pred_1,pred_cat_mean_2,pred_cat_mean_8,item_pred_cat_cos,item_price_level+star_property_min_pred_1,item_pred_ppt_cos,item_cat_vec_10,pred_cat_mean_19,item_cat_vec_13,shop_brand_

In [3]:
test = train[train.day == 25]
val = train[train.day == 24]
train = train[(train.day != 24) & (train.day != 25)]

train.shape
val.shape
test.shape

(203112, 266)

(57411, 266)

(18371, 266)

In [4]:
import sys
sys.path.append('/Users/zero/xgboost/python-package')
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
predictors = [
    'user_query_best_service', 'pred_cat_mean_4', 'user_day_query',
    'pred_property_split_count', 'user_sell_trade_power',
    'item_collected_level+user_gender_id_pred_1', 'item_cat_vec_22',
    'shop_score_delivery', 'item_score2', '24h_user_seem_times',
    'gender_filled', 'user_star_level', 'pred_cat_mean_0', '24h_cat_hot',
    'user_sell_power', 'shop_review_num_level', 'brand_hot', 'pred_cat_mean_1',
    'pred_cat_mean_7', 'item_cat_vec_2',
    'item_brand_id+shop_score_description_pred_1', 'is_home',
    'item_pred_property_score', 'shop_score_service', 'shop_score',
    'shop_item_cnt+shop_brand_cnt_pred_1', 'item_pred_property_score_pred%',
    'item_cat_vec_16', 'user_minute_query', 'minute',
    'item_category_split_count', 'brand_is_trade_his',
    'shop_review_num_level+hist_hour_ctr_pred_1', 'day',
    'item_sales_level+item_collected_level_pred_1', 'item_id_pred_1',
    'item_pred_property_score_item%', 'user_query_cheapest',
    'city_brand_count_his', 'is_last_click', 'item_cat_vec_12',
    'user_sell_power_mean_item_minus', 'user_query_maxreview',
    'user_item_count', 'max_click_time_sub', 'item_cat_vec_18', 'item_hot',
    'user_shop_last_time_sub', 'item_brand_id+star_property_max_pred_1',
    'user_shop_count', 'shop_star_level', 'pred_cat_mean_18',
    'pred_cat_mean_6', 'user_hot', 'star_property_max', 'shop_brand_cnt',
    'item_brand_id+item_price_level_pred_1', 'item_cat_vec_9',
    'item_city_id+context_page_id_pred_1', 'user_age_level+user_id_pred_1',
    'user_shop_day_num', 'user_context_trade_count', 'pred_cat_mean_14',
    'item_sales_level+item_pv_level_pred_1', 'item_pred_category_score',
    'user_gender_id+star_category_min_pred_1', 'item_cat_vec_4',
    'user_item_next_time_sub', 'pred_cat_mean_15', 'user_shop_num',
    'item_cat_vec_0', 'item_collected_level+shop_score_description_pred_1',
    'user_item_hour_query', 'occupation_trade_hot', 'pred_cat_mean_13',
    'item_id+hist_hour_ctr_pred_1', 'user_hour_count',
    'max_user_item_click_time_sub', 'user_query_maxqueryitem_p_similarity',
    'pred_cat_mean_12', 'user_gender_id', 'is_last_user_item_click',
    'item_is_trade_his', 'pred_cat_mean_3', 'item_cat_vec_1',
    'pred_cat_mean_17', 'item_price_level_pred_1', 'item_cat_vec_10',
    'pred_cat_mean_21', 'shop_hot', 'item_cat_vec_23', 'user_sell_power_mean',
    'item_price_stair', 'star_category_min', 'pred_cat_mean_11',
    'item_pv_level+shop_review_positive_rate_pred_1', 'user_is_trade_his',
    'user_item_last_time_sub', 'item_cat_vec_5', 'item_cat_vec_17',
    'user_minute_count', 'item_sales_stair', 'star_category_var',
    'star_property_min', 'user_query_maxqueryitem_c_similarity',
    'pred_cat_mean_16', 'occupation_trade_percent', 'item_sales_level',
    'item_cat_vec_13', 'pred_cat_mean_5', 'pred_cat_mean_19',
    'item_cat_vec_21', 'user_item_minute_query', 'last_time_sub',
    'item_trade_percent', 'item_pv_level', 'star_property_var',
    'occupation_hot', 'shop_score_description', 'item_cat_vec_14',
    'item_city_id_idx', 'item_occupation_trade_count',
    'user_sell_trade_power_mean_item_minus', 'context_page_id',
    'city_trade_hot', 'item_cat_vec_3', 'user_occupation_stair', 'city_hot',
    '24h_user_seem_times_pred_1', 'pred_category_split_count',
    'user_shop_count_first_time_sub', 'user_query_maxshopstar',
    'item_cat_vec_8', 'pred_cat_mean_10', 'item_brand_id+hist_hour_ctr_pred_1',
    'item_minute_query', 'item_cat_vec_20', 'user_item_trade_count',
    'item_price_level+item_pred_cat_cos_pred_1', 'city_item_count',
    'brand_is_his', 'user_age_stair', 'item_price_level', 'item_cat_vec_19',
    'user_count_first_time_sub', 'shop_review_positive_rate',
    'item_collected_level+item_id_pred_1', 'user_shop_trade_count',
    'item_cat_vec_15', 'item_pred_category_score_item%',
    'shop_star_level+shop_item_cnt_pred_1', 'user_item_num', 'pred_cat_mean_2',
    'user_brand_count', 'user_age_level+user_star_level_pred_1', 'item_is_his',
    'user_context_count', 'pred_cat_mean_22', 'hour_stair',
    'user_star_level+shop_id_pred_1', 'city_item_count_his',
    'user_query_maxsell', 'pred_cat_mean_23', 'user_yesterday_query',
    'item_cat_vec_7', 'hour', 'brand_item_count_his', 'item_cat_vec_6',
    'user_query_best_delivery', 'pred_cat_mean_20', 'item_collected_level',
    'user_query_best_description', 'user_count',
    'shop_review_num_level+shop_item_cnt_pred_1', 'user_shop_next_time_sub',
    'item_cat_vec_11', 'yesterday', 'user_occupation_id', 'user_day_count',
    'item_property_split_count', 'pred_cat_mean_9', 'pred_cat_mean_8',
    'user_hour_query', 'context_page_stair', 'shop_item_count', '24h_ppt_hot',
    'shop_item_cnt+star_property_var_pred_1', 'user_query_maxcollect',
    'shop_id_pred_1', 'user_brand_trade_count', 'yesterhour', 'user_is_his',
    'shop_item_cnt', 'item_brand_id+shop_brand_cnt_pred_1', 'user_age_level',
    'star_category_max', 'user_item_count_first_time_sub', 'user_star_stair',
    'next_time_sub', 'city_brand_count', 'user_item_day_num',
    'user_query_maxpv', 'shop_trade_hot', 'star_property_mean',
    'user_trade_hot'
]

target = ['is_trade']

In [7]:
field_predictors = []
for i in predictors:
    if ('pred_cat_mean' in i) | ('item_cat_ve' in i):
        field_predictors.append(i)
len(field_predictors)

one_hot = [
    'brand_is_his', 'brand_is_trade_his', 'context_page_stair',
    'gender_filled', 'hour_stair', 'is_home', 'is_last_click',
    'is_last_user_item_click', 'item_city_id_idx', 'item_is_his',
    'item_is_trade_his', 'item_price_stair', 'item_sales_stair',
    'user_age_stair', 'user_gender_id', 'user_is_his', 'user_is_trade_his',
    'user_occupation_id', 'user_occupation_stair', 'user_query_best_delivery',
    'user_query_best_description', 'user_query_best_service',
    'user_query_cheapest', 'user_query_maxcollect', 'user_query_maxpv',
    'user_query_maxqueryitem_c_similarity',
    'user_query_maxqueryitem_p_similarity', 'user_query_maxreview',
    'user_query_maxsell', 'user_query_maxshopstar', 'user_star_stair'
]
len(one_hot)

predictors = list(set(predictors) - set(field_predictors) - set(one_hot))

48

31

In [18]:
#train/val
params = {
    'objective': 'binary:logistic',
    'eta': 0.15,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 3,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    #     'early_stopping_rounds' : 50,
    'missing': -1
}

len(predictors)
xgbtrain = xgb.DMatrix(train[predictors], train[target])
xgbval = xgb.DMatrix(val[predictors])
model = xgb.train(params, xgbtrain, num_boost_round=300)
y_pred = model.predict(xgbval)
y_pred = pd.Series(y_pred, name='val')

print(log_loss(val[target], y_pred))
# 0.0791929093708 0.06 * 500
# 0.0793475868447 0.1 * 250
# 0.079447 0.15 * 200
# 0.0793017175577 0.15 * 250
# 0.0795068179083 0.1 * 200

# 0.0791221135217 0.15 * 750

# ---------157------------
# 0.0794218275816 0.15 * 300
# 0.079524718608 0.15 * 250
# 0.0797669564197 0.15 * 250 * 4
# 0.0793976740046 0.08 * 500
# 0.0794570369009 0.06 * 500

# ----------126-----------
# 0.079508249703 0.15 * 300

126

0.0795856290706


In [8]:
params = {
    'objective': 'binary:logistic',
    'eta': 0.15,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 3,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    #     'early_stopping_rounds' : 50,
    'missing': -1
}

len(predictors)
cv_train = pd.concat([train, val])
xgbtrain = xgb.DMatrix(cv_train[predictors], cv_train[target])
model = xgb.train(params, xgbtrain, num_boost_round=330)

126

In [9]:
xgbtest = xgb.DMatrix(test[predictors])

ffm_trainval = model.predict(xgbtrain, pred_leaf=True)
ffm_test = model.predict(xgbtest, pred_leaf=True)

In [10]:
ffm_trainval.shape
ffm_test.shape

(260523, 330)

(18371, 330)

In [ ]:
import ffm
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV

In [11]:
ffm_trainval_matrix = []
ffm_test_matrix = []

for i in tqdm(range(0, ffm_trainval.shape[0])):
    temp_list = []
    for j in range(0, ffm_trainval.shape[1]):
        temp = []
        temp.append(j + 1)
        temp.append(ffm_trainval[i, j])
        temp.append(1)
        temp_list.append(temp)
    ffm_trainval_matrix.append(temp_list)

for i in tqdm(range(0, ffm_test.shape[0])):
    temp_list = []
    for j in range(0, ffm_test.shape[1]):
        temp = []
        temp.append(j + 1)
        temp.append(ffm_test[i, j])
        temp.append(1)
        temp_list.append(temp)
    ffm_test_matrix.append(temp_list)

100%|██████████| 18371/18371 [00:21<00:00, 852.86it/s] 


In [ ]:
# ffm1 205 * 250
# ffm2 205 * 250(2)
# ffm3 157 * 330
# ffm4 157 * 330(2)
# ffm5 205 * 350(2)

In [15]:
# ffm1
f = open("./libffm/libffm_toy/ffm_tr.ffm", "a+")
for i in tqdm(range(0, train.shape[0])):
    tmp = f.write(str(train.is_trade[i]))
    for j in range(0, ffm_trainval.shape[1]):
        tmp = f.write(' ' + str(j + 1) + ':' + str(ffm_trainval[i, j]) + ':1')
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_tr.ffm", 'rU').readlines())

f = open("./libffm/libffm_toy/ffm_vl.ffm", "a+")
for i in tqdm(range(0, val.shape[0])):
    tmp = f.write(str(val.is_trade[train.shape[0] + i]))
    for j in range(0, ffm_trainval.shape[1]):
        tmp = f.write(' ' + str(j + 1) + ':' +
                      str(ffm_trainval[train.shape[0] + i, j]) + ':1')
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_vl.ffm", 'rU').readlines())

f = open("./libffm/libffm_toy/ffm_test.ffm", "a+")
for i in tqdm(range(0, test.shape[0])):
    for j in range(0, ffm_test.shape[1]):
        tmp = f.write(str(j + 1) + ':' + str(ffm_test[i, j]) + ':1 ')
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_test.ffm", 'rU').readlines())

100%|██████████| 203112/203112 [14:13<00:00, 238.02it/s]


203112

100%|██████████| 57411/57411 [05:26<00:00, 176.07it/s]


57411

100%|██████████| 18371/18371 [00:39<00:00, 466.37it/s]


18371

In [16]:
# ffm2
f = open("./libffm/libffm_toy/ffm_tr2.ffm", "a+")
for i in tqdm(range(0, train.shape[0])):
    tmp = f.write(str(train.is_trade[i]))
    for j in range(0, ffm_trainval.shape[1]):
        tmp = f.write(' ' + str(j + 1) + ':' + str(
            (32 * j) + ffm_trainval[i, j]) + ':1')
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_tr2.ffm", 'rU').readlines())

f = open("./libffm/libffm_toy/ffm_vl2.ffm", "a+")
for i in tqdm(range(0, val.shape[0])):
    tmp = f.write(str(val.is_trade[train.shape[0] + i]))
    for j in range(0, ffm_trainval.shape[1]):
        tmp = f.write(' ' + str(j + 1) + ':' + str(
            (32 * j) + ffm_trainval[train.shape[0] + i, j]) + ':1')
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_vl2.ffm", 'rU').readlines())

f = open("./libffm/libffm_toy/ffm_test2.ffm", "a+")
for i in tqdm(range(0, test.shape[0])):
    for j in range(0, ffm_test.shape[1]):
        tmp = f.write(
            str(j + 1) + ':' + str((32 * j) + ffm_test[i, j]) + ':1 ')
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_test2.ffm", 'rU').readlines())

100%|██████████| 203112/203112 [28:48<00:00, 117.54it/s]


203112

100%|██████████| 57411/57411 [08:37<00:00, 110.98it/s]


57411

100%|██████████| 18371/18371 [01:48<00:00, 169.76it/s]


18371

In [ ]:
# ffm4
ffm_trainval_concat = pd.concat(
    [
        pd.DataFrame(ffm_trainval),
        pd.concat([train, val])[field_predictors + one_hot]
    ],
    axis=1)
ffm_trainval_concat.shape
ffm_test_concat = pd.concat(
    [
        pd.DataFrame(ffm_test, index=test.index),
        test[field_predictors + one_hot]
    ],
    axis=1)
ffm_test_concat.shape

f = open("./libffm/libffm_toy/ffm_tr4.ffm", "a+")
for i in tqdm(range(0, train.shape[0])):
    field_max = 0
    tmp = f.write(str(train.is_trade[i]))
    for j in range(0, ffm_trainval_concat.shape[1]):
        if ffm_trainval_concat.columns[j] in field_predictors:
            tmp = f.write(' ' + str(field_max + 1) + ':' +
                          str((32 * field_max) + ffm_trainval_concat.iloc[i, j]) + ':1')
        elif ffm_trainval_concat.columns[j] in one_hot:
            tmp = f.write(' ' + str(field_max + 1) + ':' +
                          str((32 * field_max) + ffm_trainval_concat.iloc[i, j]) + ':1')
            field_max += 1
        else:
            tmp = f.write(' ' + str(field_max + 1) + ':' +
                          str((32 * field_max) + ffm_trainval_concat.iloc[i, j]) + ':1')
            field_max += 1
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_tr4.ffm", 'rU').readlines())

f = open("./libffm/libffm_toy/ffm_vl4.ffm", "a+")
for i in tqdm(range(0, val.shape[0])):
    field_max = 0
    tmp = f.write(str(val.is_trade[train.shape[0] + i]))
    for j in range(0, ffm_trainval_concat.shape[1]):
        if ffm_trainval_concat.columns[j] in field_predictors:
            tmp = f.write(' ' + str(field_max + 1) + ':' + str(
                (32 * field_max) + ffm_trainval_concat.iloc[train.shape[0] + i, j]) + ':1')
        elif ffm_trainval_concat.columns[j] in one_hot:
            tmp = f.write(' ' + str(field_max + 1) + ':' + str(
                (32 * field_max) + ffm_trainval_concat.iloc[train.shape[0] + i, j]) + ':1')
            field_max += 1
        else:
            tmp = f.write(' ' + str(field_max + 1) + ':' + str(
                (32 * field_max) + ffm_trainval_concat.iloc[train.shape[0] + i, j]) + ':1')
            field_max += 1
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_vl4.ffm", 'rU').readlines())

f = open("./libffm/libffm_toy/ffm_test4.ffm", "a+")
for i in tqdm(range(0, test.shape[0])):
    field_max = 0
    for j in range(0, ffm_test_concat.shape[1]):
        if ffm_test_concat.columns[j] in field_predictors:
            tmp = f.write(' ' + str(field_max + 1) + ':' +
                          str((32 * field_max) + ffm_test_concat.iloc[i, j]) + ':1')
        elif ffm_test_concat.columns[j] in one_hot:
            tmp = f.write(' ' + str(field_max + 1) + ':' +
                          str((32 * field_max) + ffm_test_concat.iloc[i, j]) + ':1')
            field_max += 1
        else:
            tmp = f.write(
                str(field_max + 1) + ':' + str((32 * field_max) + ffm_test_concat.iloc[i, j]) +
                ':1 ')
            field_max += 1
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_test4.ffm", 'rU').readlines())

(260523, 409)

(18371, 409)

  1%|          | 2165/203112 [19:49<30:40:30,  1.82it/s]

In [ ]:
# ffm4
ffm_trainval_concat = pd.concat(
    [
        pd.DataFrame(ffm_trainval),
        pd.concat([train, val])[field_predictors + one_hot]
    ],
    axis=1)
ffm_trainval_concat.shape
ffm_test_concat = pd.concat(
    [
        pd.DataFrame(ffm_test, index=test.index),
        test[field_predictors + one_hot]
    ],
    axis=1)
ffm_test_concat.shape

field_max = 0
f = open("./libffm/libffm_toy/ffm_tr4.ffm", "a+")
for i in tqdm(range(0, train.shape[0])):
    tmp = f.write(str(train.is_trade[i]))
    for j in range(0, ffm_trainval_concat.shape[1]):
        if ffm_trainval_concat.columns[j] in field_predictors:
            tmp = f.write(' ' + str(field_max + 1) + ':' +
                          str(ffm_trainval_concat.iloc[i, j]) + ':1')
        elif ffm_trainval_concat.columns[j] in one_hot:
            tmp = f.write(' ' + str(field_max + 1) + ':' +
                          str(ffm_trainval_concat.iloc[i, j]) + ':1')
            field_max += 1
        else:
            tmp = f.write(' ' + str(field_max + 1) + ':' +
                          str(ffm_trainval_concat.iloc[i, j]) + ':1')
            field_max += 1
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_tr4.ffm", 'rU').readlines())

field_max = 0
f = open("./libffm/libffm_toy/ffm_vl4.ffm", "a+")
for i in tqdm(range(0, val.shape[0])):
    tmp = f.write(str(val.is_trade[train.shape[0] + i]))
    for j in range(0, ffm_trainval_concat.shape[1]):
        if ffm_trainval_concat.columns[j] in field_predictors:
            tmp = f.write(' ' + str(field_max + 1) + ':' + str(
                ffm_trainval_concat.iloc[train.shape[0] + i, j]) + ':1')
        elif ffm_trainval_concat.columns[j] in one_hot:
            tmp = f.write(' ' + str(field_max + 1) + ':' + str(
                ffm_trainval_concat.iloc[train.shape[0] + i, j]) + ':1')
            field_max += 1
        else:
            tmp = f.write(' ' + str(field_max + 1) + ':' + str(
                ffm_trainval_concat.iloc[train.shape[0] + i, j]) + ':1')
            field_max += 1
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_vl4.ffm", 'rU').readlines())

field_max = 0
f = open("./libffm/libffm_toy/ffm_test4.ffm", "a+")
for i in tqdm(range(0, test.shape[0])):
    for j in range(0, ffm_test_concat.shape[1]):
        if ffm_test_concat.columns[j] in field_predictors:
            tmp = f.write(' ' + str(field_max + 1) + ':' +
                          str(ffm_test_concat.iloc[i, j]) + ':1')
        elif ffm_test_concat.columns[j] in one_hot:
            tmp = f.write(' ' + str(field_max + 1) + ':' +
                          str(ffm_test_concat.iloc[i, j]) + ':1')
            field_max += 1
        else:
            tmp = f.write(
                str(field_max + 1) + ':' + str(ffm_test_concat.iloc[i, j]) +
                ':1 ')
            field_max += 1
    tmp = f.write('\n')
f.close()
len(open("./libffm/libffm_toy/ffm_test4.ffm", 'rU').readlines())

In [12]:
ffm_trainval_matrix = np.array(ffm_trainval_matrix)
ffm_test_matrix = np.array(ffm_test_matrix)

ffm_trainval_matrix.shape
ffm_test_matrix.shape

(260523, 250, 3)

(18371, 250, 3)

In [13]:
np.save('ffm_trainval_matrix2.npy', ffm_trainval_matrix)
np.save('ffm_test_matrix2.npy', ffm_test_matrix)

In [103]:
ffm_trainval_matrix = np.load('ffm_trainval_matrix2.npy')
ffm_test_matrix = np.load('ffm_test_matrix2.npy')

In [14]:
ffm_trainval_y = np.array(cv_train[target])
ffm_trainval = ffm.FFMData(ffm_trainval_matrix, ffm_trainval_y)
ffm_test = ffm.FFMData(ffm_test_matrix)

ffm_train = ffm.FFMData(ffm_trainval_matrix[:-57411, :],
                        ffm_trainval_y[:-57411])
ffm_val = ffm.FFMData(ffm_trainval_matrix[-57411:, :], ffm_trainval_y[-57411:])

In [23]:
ffm_model = ffm.FFM(eta=0.25, lam=0.0001, k=4)
ffm_model.fit(
    ffm_train,
    val_data=ffm_val,
    num_iter=30,
    metric='auc',
    early_stopping=5,
    maximum=True)

Iter    Train_Loss      Train_Score     Val_Score       Best_Iter


KeyboardInterrupt: 

In [99]:
params = {
    'eta': [0.1, 0.3],
    'lam': [0.0001],
    'k': [4],
    'num_iter': [30],
    'early_stopping': [5],
    'metric': ['logloss']
}

ffm_model = ffm.FFM()

ffm_grid = GridSearchCV(
    ffm_model, params, verbose=1, cv=5, n_jobs=-1, scoring='neg_log_loss')
ffm_grid.fit(ffm_trainval_matrix)
ffm_grid.best_params_
ffm_grid.best_score_

TypeError: Singleton array array(<ffm.ffm.FFMData object at 0x20cbf38b00>, dtype=object) cannot be considered a valid collection.